<div style="text-align: center; font-size: 16px;">
    <strong>Course:</strong> Machine Learning Operations |
    <strong>Lecturer:</strong> Prof. Dr. Klotz |
    <strong>Date:</strong> 17.05.2025 |
    <strong>Name:</strong> Sofie Pischl
</div>

# <center> Data Exploration </center>

# Inhaltsverzeichnis
1. [Einleitung](#Einleitung)
2. [Datenvorbereitung](#Datenvorbereitung)
3. [Fortgeschrittenes Topic Modelling mit BERTopic](#Fortgeschrittenes-Topic-Modelling-mit-BERTopic)
4. [Verbesserte Sentiment-Analyse](#Verbesserte-Sentiment-Analyse)
5. [Zeitliche Analyse der Topics](#Zeitliche-Analyse-der-Topics)
6. [UMAP-Visualisierung der Topics](#UMAP-Visualisierung-der-Topics)
7. [Vorbereitung auf Klassifikation](#Vorbereitung-auf-Klassifikation)


## Einleitung
Dieses Notebook untersucht kombinierte Social-Media-Daten aus Reddit, YouTube und TikTok. Ziel ist es, Muster in Themen und Stimmungen zu erkennen und die Daten für eine spätere Klassifikation vorzubereiten. Wir verwenden moderne Methoden wie **BERTopic**, **Transformer-basierte Sentimentanalyse** und **visuelle Embedding-Cluster**.

In diesem Notebook bauen wir auf der bisherigen Datenexploration auf und ergänzen folgende moderne Verfahren:

1. **Fortgeschrittenes Topic Modelling mit BERTopic**
2. **Verbesserte Sentiment-Analyse mit VADER und Transformer-Modell**
3. **Detaillierte Exploration von Text-Clustern und Zeitreihen**
4. **Vorbereitung für Klassifikation und Vorhersagemodelle**

Jede Methode wird durch beschreibenden Text begleitet und in das bestehende Datenformat integriert.

## Datenvorbereitung
Wir laden die vorverarbeiteten Daten, konvertieren Zeitstempel und bereiten die Texte für die Analyse vor.

In [22]:
import pandas as pd
pd.set_option('display.max_columns', 1000)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import nltk
from pathlib import Path

In [23]:
BASE_DIR = Path().resolve().parent
RAW_DIR = (BASE_DIR / "./data/processed").resolve()

# Datei laden
df = pd.read_csv(RAW_DIR / "social_media_data.csv")

display(df.head())
df.info()

source                   id title  \
0  tiktok  7492000423641959685   NaN   
1  tiktok  7461927005689302280   NaN   
2  tiktok  7461757350492278048   NaN   
3  tiktok  7462429961345879318   NaN   
4  tiktok  7477658166235286791   NaN   

                                                text               username  \
0  #CapCut #قوالب_كاب_كات_جاهزه_للتصميم__🌴♥ #كاب_...   noordeen_cap_cat_0_1   
1  welcome to the thanos world!! #squidgame #squi...  team_thanos_player230   
2  This is a great goal 😳😭 @Fabio Andrés #golazo ...          risingballers   
3  glaube er hatte nicht so bock auf mich..💀 #for...             allroundii   
4  #mascotas #humormascotas😂😂 #mascotastiktok #vi...            rokopitbull   

      likes  comments    shares     plays   timestamp         published_at  \
0  0.001497  0.000989  0.000181  0.003962  1744367290  2025-05-12 13:56:20   
1  0.006703  0.028503  0.001095  0.037793  1737365272  2025-05-12 13:56:20   
2  0.022272  0.012754  0.026598  0.033222  1737325774  2025-05-12 13:56:20   
3  0.017508  0.015076  0.018989  0.031088  1737482377  2025-05-12 13:56:20   
4  0.387179  0.085289  0.252874  0.310576  1741027969  2025-05-12 13:56:20   

                                                 url title_language  \
0  https://webapp-va.tiktok.com/6508d64d970e751a2...        unknown   
1  https://webapp-sg.tiktok.com/bf63b8aa40b9ff0ca...        unknown   
2  https://v16-webapp-prime.tiktok.com/video/tos/...        unknown   
3                                                NaN        unknown   
4  https://v16-webapp-prime.tiktok.com/video/tos/...        unknown   

  title_clean  title_char_count  title_word_count  title_uppercase_count  \
0         NaN               0.0               0.0                    0.0   
1         NaN               0.0               0.0                    0.0   
2         NaN               0.0               0.0                    0.0   
3         NaN               0.0               0.0                    0.0   
4         NaN               0.0               0.0                    0.0   

   title_exclamation_count  title_question_count  title_emoji_count  \
0                      0.0                   0.0                0.0   
1                      0.0                   0.0                0.0   
2                      0.0                   0.0                0.0   
3                      0.0                   0.0                0.0   
4                      0.0                   0.0                0.0   

   title_mention_count  title_hashtag_count  title_avg_word_length  \
0                  0.0                  0.0                    0.0   
1                  0.0                  0.0                    0.0   
2                  0.0                  0.0                    0.0   
3                  0.0                  0.0                    0.0   
4                  0.0                  0.0                    0.0   

  title_sentiment  title_sentiment_score text_language            text_clean  \
0         neutral                    0.0       unknown                   NaN   
1         neutral                    0.0            en  welcome thanos world   
2         neutral                    0.0            en     great goal andrés   
3         neutral                    0.0            de           glaube bock   
4         neutral                    0.0       unknown                   NaN   

   text_char_count  text_word_count  text_uppercase_count  \
0              0.0              0.0                   0.0   
1             20.0              3.0                   0.0   
2             17.0              3.0                   0.0   
3             11.0              2.0                   0.0   
4              0.0              0.0                   0.0   

   text_exclamation_count  text_question_count  text_emoji_count  \
0                     0.0                  0.0               0.0   
1                     0.0                  0.0               0.0   
2                     0.0         

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2205 entries, 0 to 2204
Data columns (total 45 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   source                   2205 non-null   object 
 1   id                       2205 non-null   object 
 2   title                    778 non-null    object 
 3   text                     2204 non-null   object 
 4   username                 1821 non-null   object 
 5   likes                    2205 non-null   float64
 6   comments                 2205 non-null   float64
 7   shares                   2205 non-null   float64
 8   plays                    2205 non-null   float64
 9   timestamp                2205 non-null   object 
 10  published_at             2186 non-null   object 
 11  url                      1257 non-null   object 
 12  title_language           2205 non-null   object 
 13  title_clean              777 non-null    object 
 14  title_char_count        

In [24]:
# Anzahl der Posts pro Plattform
post_counts = df["source"].value_counts().reset_index()
post_counts.columns = ["source", "num_posts"]

# Anzeige
print(post_counts)

    source  num_posts
0   tiktok       1427
1  youtube        394
2   reddit        384


# Preparations

In [25]:
# Kombinieren von Titel und Text (beide bereinigt)
texts = (df["title_clean"].fillna("") + " " + df["text_clean"].fillna("")).astype(str)

# LDA
LDA ist ein probabilistisches generatives Modell, das davon ausgeht, dass jedes Dokument eine Mischung aus mehreren Themen ist und jedes Thema durch eine Verteilung über Wörter beschrieben werden kann. Es eignet sich gut für längere, formellere Texte, bietet jedoch auch bei kurzen Texten erste Einblicke in thematische Strukturen.

Warum LDA als erster Ansatz?
Einfachheit & Transparenz: LDA ist leicht verständlich und bietet eine gute Grundlage, um Themenmodelle zu interpretieren.

Vergleichsbasis: Es dient als Referenzmodell, gegen das modernere Verfahren wie BERTopic verglichen werden können.

Schnelle Anwendung: Mit scikit-learn lässt sich LDA effizient auf vorbereiteten TF-IDF-Daten trainieren

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# CountVectorizer ohne Stopwords (da vorher entfernt)
vectorizer = CountVectorizer(
    max_df=0.95,
    min_df=2
)

# Textmatrix erzeugen
X = vectorizer.fit_transform(texts)  # texts = bereits bereinigte Texte

# LDA-Modell definieren
lda_model = LatentDirichletAllocation(
    n_components=9,         # Anzahl der Themen
    max_iter=3,
    learning_method='online',
    random_state=42
)

# LDA-Modell trainieren
lda_model.fit(X)


LatentDirichletAllocation(learning_method='online', max_iter=3, n_components=9,
                          random_state=42)

In [27]:
# Funktion zum Anzeigen der Themen mit Top-Wörtern
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"🟢 Thema {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        print()

display_topics(lda_model, vectorizer.get_feature_names_out(), 10)


🟢 Thema 1:
instagram youtube video league de dazn facebook twitter vs highlights

🟢 Thema 2:
time like game life go work new love since baby

🟢 Thema 3:
people pf official two part pts pm different nbsp stl

🟢 Thema 4:
real support world months link guys summer content story performance

🟢 Thema 5:
said even make right people say see let music better

🟢 Thema 6:
like one would get know told got back could want

🟢 Thema 7:
media oblivion vi funk social voice papst leben elder kids

🟢 Thema 8:
wig last weekend wife fox gift style party mom posted

🟢 Thema 9:
tiktok mehr folge geht instagram beim immer facebook findet by



## 1. Fortgeschrittenes Topic Modelling mit BERTopic

Das klassische LDA-Verfahren ist oft ungeeignet für kurze, informelle Texte. BERTopic nutzt Embeddings + Clustering + TF-IDF, um deutlich robustere und interpretierbare Themen zu erzeugen. Zudem unterstützt BERTopic die Analyse von Themen im Zeitverlauf.

In [15]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

# Für deutschsprachige Texte kann ein mehrsprachiges Modell hilfreich sein
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # oder 'paraphrase-multilingual-MiniLM-L12-v2'

# Daten vorbereiten
texts = df['text_clean'].dropna().tolist()

# BERTopic initialisieren und trainieren
topic_model = BERTopic(embedding_model=embedding_model, language='multilingual')
topics, probs = topic_model.fit_transform(texts)

df_clean = df.loc[df['text_clean'].notna()].copy()
df_clean['topic'] = topics

topic_model.get_topic_info()

Topic  Count                                     Name  \
0      -1    553             -1_music_instagram_max_video   
1       0    317                   0_said_like_told_would   
2       1    200           1_instagram_video_tiktok_kanal   
3       2    180                2_wait_bro_welcome_summer   
4       3     60              3_than_einfach_ist_stronger   
5       4     49                           4_que_de_em_os   
6       5     48                       5_daw_naman_mga_sa   
7       6     45          6_khan_account_unfreez_official   
8       7     45                        7_pf_pts_nbsp_ast   
9       8     44           8_league_dazn_highlights_sport   
10      9     43                         9_في_على_من_اللي   
11     10     34                  10_hair_bio_makeup_asmr   
12     11     31                          11_ik_de_ze_het   
13     12     30          12_ronaldo_shape_football_messi   
14     13     28                     13_yamal_grwm_mgk_jf   
15     14     26               14_battle_new_players_play   
16     15     20        15_nurburgring_turbo_bmw_mercedes   
17     16     20  16_production_azizam_assistant_director   
18     17     19                 17_cat_kitten_cats_puppy   
19     18     17          18_viral_duolingo_tiktok_rumble   
20     19     16          19_brainrot_tragic_italian_song   
21     20     14           20_different_bars_et_chocolate   
22     21     14                 21_song_mona_motion_lisa   
23     22     13           22_oblivion_scrolls_elder_game   
24     23     13                  23_на_кисело_тунг_мляко   
25     24     12              24_prank_fart_anatoly_spray   
26     25     11             25_tung_sahur_nela_ballerina   
27     26     11             26_template_new_capcut_edits   
28     27     11   27_これめっちゃ頭痛くなる_公開_全国劇場にて_またすぐにお会いしましょう   

                                       Representation  \
0   [music, instagram, max, video, follow, youtube...   
1   [said, like, told, would, time, even, people, ...   
2   [instagram, video, tiktok, kanal, youtube, wer...   
3   [wait, bro, welcome, summer, love, part, vibe,...   
4   [than, einfach, ist, stronger, immer, polizei,...   
5   [que, de, em, os, não, para, quando, todos, ou...   
6   [daw, naman, mga, sa, namimilit, meu, yıldız, ...   
7   [khan, account, unfreez, official, część, typp...   
8   [pf, pts, nbsp, ast, blk, stl, min, reb, team,...   
9   [league, dazn, highlights, sport, uefa, sky, c...   
10  [في, على, من, اللي, انا, لا, الدنيا, official,...   
11  [hair, bio, makeup, asmr, mascara, blush, wash...   
12    [ik, de, ze, het, zijn, en, van, dat, nie, als]   
13  [ronaldo, shape, football, messi, goal, skills...   
14  [yamal, grwm, mgk, jf, lamine, letter, big, si...   
15  [battle, new, players, play, support, free, cl...   
16  [nurburgring, turbo, bmw, mercedes, bi, amg, g...   
17  [production, azizam, assistant, director, na, ...   
18  [cat, kitten, cats, puppy, diaper, belly, danc...   
19  [viral, duolingo, tiktok, rumble, tiktoks, pho...   
20  [brainrot, tragic, italian, song, speedrun, mo...   
21  [different, bars, et, chocolate, al, zucker, p...   
22  [song, mona, motion, lisa, plattforms, moments...   
23  [oblivion, scrolls, elder, game, remastered, i...   
24  [на, кисело, тунг, мляко, меня, да, ток, проте...   
25  [prank, fart, anatoly, spray, replying, pranks...   
26  [tung, sahur, nela, ballerina, brainrot, overs...   
27  [template, new, capcut, edits, backup, try, mo...   
28  [これめっちゃ頭痛くなる, 公開, 全国劇場にて, またすぐにお会いしましょう, 劇場版, ...   

                                  Representative_Docs  
0   [go dive deeper topics free day trial premium ...  
1   [f three children ex kon months ago sister f c...  
2   [folgt instagram twitch twitter youtube main k...  
3   [beluga lagoon the glen such a summer vibe ban...  
4   [gilts mindestens zwei freunde mitnehmen wahll...  
5   [que encontro maravilhoso provando que os quad...  
6   [insta ahmad neu, terlalu pagi muka bengkak pl...  
7   [unfreez a

In [16]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN
import nltk
nltk.download('stopwords')  # Nur beim ersten Mal nötig

from nltk.corpus import stopwords

# Schritt 1: Besseres Embedding-Modell (wenn mehrsprachig)
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

custom_stopwords = ['like', 'said', 'instagram', 'video', 'follow', 'people', 'know', 'thing', 'tiktok', 'every', 'max']

# Schritt 2: Besserer Vectorizer
vectorizer_model = CountVectorizer(
    stop_words=custom_stopwords + stopwords.words("english"),
    ngram_range=(1, 2),
    min_df=2
)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SofiePischl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:

# Schritt 3: Optional – feinjustierter Clusterer
hdbscan_model = HDBSCAN(min_cluster_size=12, min_samples=1,  metric='euclidean',
    cluster_selection_method='eom')

# Schritt 4: Topic-Modellierung
topic_model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    hdbscan_model=hdbscan_model,
    min_topic_size=3,
     top_n_words=10,
    language='multilingual'
)


In [18]:

# Schritt 5: Training
texts = df['text_clean'].dropna().tolist()
topics, probs = topic_model.fit_transform(texts)

# Schritt 6: Optionale bessere Labels
topic_model.generate_topic_labels(nr_words=5, topic_prefix=False)

df_en = df[df['text_language'] == 'en']

# Schritt 7: Ergebnisse zurückschreiben
df_clean = df.loc[df['text_clean'].notna()].copy()
df_clean['topic'] = topics


In [19]:
topic_info = topic_model.get_topic_info()
display(topic_info)

Topic  Count                                       Name  \
0      -1    562                   -1_de_love_different_one   
1       0    130                   0_told_would_got_husband   
2       1     76                          1_jest_tak_tym_że   
3       2     60                    2_funk_mehr_geht_gaming   
4       3     60                    3_work_company_unity_ai   
5       4     56            4_league_dazn_sports_highlights   
6       5     54          5_antwort_description_letter_sign   
7       6     41          6_tung_tung tung_sahur_tung sahur   
8       7     41             7_trump_country_maga_americans   
9       8     40                                      8____   
10      9     39                    9_omg_going_go_go ahead   
11     10     39                         10_على_لا_que_naja   
12     11     39  11_assistant_production_producer_director   
13     12     35                12_template_hair_new_makeup   
14     13     35              13_brooks_football_ball_shape   
15     14     32             14_song_brainrot_movie_tralala   
16     15     32                         15_min_stats_ft_sf   
17     16     30          16_twitter_twitch_discord_youtube   
18     17     27     17_speedrun_mona lisa_lisa motion_mona   
19     18     26               18_schnelle_ml_einfach_schön   
20     19     26                    19_cbs_news_actors_gala   
21     20     24             20_summer_ready_morning_season   
22     21     22                   21_shakira_neu_insta_lia   
23     22     21                    22_battle_wwe_new_clash   
24     23     20                  23_funny_smile_laugh_clip   
25     24     19                     24_اللي_merlin_dino_من   
26     25     18             25_links_code_rabatt_provision   
27     26     17        26_love_love new_fell love_probiert   
28     27     17                 27_italian_em_brainrot_que   
29     28     16                  28_jess_invited_gift_apps   
30     29     16                   29_feet_gym_reaction_leg   
31     30     16                  30_cats_belly_funny_anime   
32     31     16                  31_aura_vibes_magic_geist   
33     32     16                32_support_game_jason_crowd   
34     33     15             33_mögen_first line_lock_signs   
35     34     15                 34_foreigner_leo_among_dad   
36     35     15            35_ms_passenger_beautiful_david   
37     36     15                  36_fail_bmw_julian_videos   
38     37     14                      37_turbo_amg_bi_sowie   
39     38     14                38_για_hungry_bananas_είναι   
40     39     14                  39_gelernt_boy_kid_válasz   
41     40     14                     40_music_base_und_arda   
42     41     14                  41_baby_singing_voice_top   
43     42     14                42_last one_title_last_done   
44     43     13                43_klaas_prosieben_joyn_uhr   
45     44     13                         44_și_moon_un_mein   
46     45     12               45_ok_yeah_good good_natural   
47     46     12    46_oblivion_elder scrolls_scrolls_elder   
48     47     12      47_welcome_danke_großes danke_blessed   

                                       Representation  \
0   [de, love, different, one, chocolate, better, ...   
1   [told, would, got, husband, even, family, home...   
2     [jest, tak, tym, że, ze, nie, ale, po, na, bad]   
3   [funk, mehr, geht, gaming, kanal, beste, wer, ...   
4   [work, company, unity, ai, boss, email, time, ...   
5   [league, dazn, sports, highlights, sport, sky,...   
6   [antwort, description, letter, sign, remember,...   
7   [tung, tung tung, sahur, tung sahur, ballerina...   
8   [trump, country, maga, americans, fuck, public...   
9                                [, , , , , , , , , ]   
10  [omg, going, go, go ahead, passing, schlau, ef...   
11  [على, لا, que, naja, انت, não, يا, من, att, ei...   
12  [assistant, production, producer, director, pr...   
13  [template, hair, new, makeup, try new, girl, e...  

In [20]:
import re

topic_info = topic_model.get_topic_info()

def is_junk_topic(row):
    rep = " ".join(row['Representation']) if isinstance(row['Representation'], list) else str(row['Representation'])
    # Nur Sonderzeichen oder zu kurz
    return (
        len(rep) < 10 or
        re.fullmatch(r'[\W\d\s,]+', rep) is not None or
        len(rep.split()) < 2  # z. B. nur ein Token
    )

# Liste schlechter Topics
junk_topics = topic_info[topic_info.apply(is_junk_topic, axis=1)]['Topic'].tolist()

df_filtered = df_clean[~df_clean['topic'].isin(junk_topics)]

In [21]:
texts_filtered = df_filtered['text_clean'].tolist()

from bertopic import BERTopic

topic_model_filtered = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    hdbscan_model=hdbscan_model,
    min_topic_size=5,
    top_n_words=10,
    language="multilingual"
)

topics_filtered, _ = topic_model_filtered.fit_transform(texts_filtered)

topic_model_filtered.get_topic_info()

Topic  Count                                   Name  \
0      -1    555                  -1_de_time_money_back   
1       0    103               0_mehr_code_rabatt_dabei   
2       1    102              1_told_husband_kids_would   
3       2     71                      2_tak_jest_tym_ze   
4       3     60                    3_work_unity_ai_get   
5       4     57  4_music_production_assistant_producer   
6       5     55      5_tung_tung tung_sahur_tung sahur   
7       6     52        6_league_dazn_sports_highlights   
8       7     45          7_schnelle_ml_einfach_schnell   
9       8     43         8_trump_country_americans_fuck   
10      9     41                      9_على_لا_naja_انت   
11     10     39            10_brooks_via_ball_football   
12     11     38    11_summer_shake_ready_getting ready   
13     12     35            12_template_hair_new_makeup   
14     13     35           13_oblivion_battle_elder_new   
15     14     32                     14_min_stats_ft_sf   
16     15     31                    15_ups_ok_cool_yeah   
17     16     30           16_yamal_foreigner_among_neu   
18     17     27               17_prank_funny_smile_que   
19     18     25                18_cbs_news_actors_gala   
20     19     24                   19_merlin_dino_ne_te   
21     20     24        20_twitch_twitter_discord_kanal   
22     21     23            21_chef_hallo_welcome_danke   
23     22     20        22_coworker_dating_jess_invited   
24     23     19     23_antwort_description_belong_quoi   
25     24     18          24_podcast_musik_werbung_link   
26     25     18           25_mini_mögen_athena_sverige   
27     26     17    26_ms_passenger_attractive_stunning   
28     27     17        27_barça_fast_geworden_stopping   
29     28     17              28_fail_bmw_julian_videos   
30     29     16           29_feet_reaction gym_leg_gym   
31     30     16       30_love_love love_love new_blank   
32     31     16              31_cats_belly_funny_anime   
33     32     15            32_support_game_crowd_jason   
34     33     15      33_definitiv_válasz_részére_piace   
35     34     15     34_going_stay safe_passing_efforts   
36     35     14            35_klaas_prosieben_joyn_uhr   
37     36     14             36_nem_fehlt_via_bold ever   
38     37     14              37_baby_singing_voice_top   
39     38     14                 38_barça_bock_vini_nom   
40     39     13                  39_turbo_bi_amg_sowie   
41     40     13              40_moments_best_face_card   
42     41     12       41_schlau_go ahead_gern_vacation   
43     42     12         42_song_brainrot_movie_tralala   
44     43     12                   43_sound_dj_ganz_mal   

                                       Representation  \
0   [de, time, money, back, would, one, got, first...   
1   [mehr, code, rabatt, dabei, kanal, facebook, l...   
2   [told, husband, kids, would, family, want, ask...   
3   [tak, jest, tym, ze, nie, po, ale, że, aura, bad]   
4   [work, unity, ai, get, day, even, company, ema...   
5   [music, production, assistant, producer, direc...   
6   [tung, tung tung, sahur, tung sahur, ballerina...   
7   [league, dazn, sports, highlights, sport, sky,...   
8   [schnelle, ml, einfach, schnell, schön, el, cl...   
9   [trump, country, americans, fuck, public, news...   
10     [على, لا, naja, انت, ou, يا, من, att, que, de]   
11  [brooks, via, ball, football, shape, messi, ro...   
12  [summer, shake, ready, getting ready, morning,...   
13  [template, hair, new, makeup, try new, girl, b...   
14  [oblivion, battle, elder, new, game, wwe, iv, ...   
15  [min, stats, ft, sf, sg, team, den, pm, game, ...   
16  [ups, ok, cool, yeah, dnia, μας, outrageous, ε...   
17  [yamal, foreigner, among, neu, dad, insta, lam...   
18  [prank, funny, smile, que, fart, spray, laugh,...   
19  [cbs, news, actors, gala, met gala, outfit, fa...   
20  [merlin, dino, ne, te, mi, ko, kenza, rocky, v...   
21  [twitch, twitter, discord, kana

In [ ]:
topic_model.visualize_documents(texts)

In [32]:
!ollama run mistral

'ollama' is not recognized as an internal or external command,
operable program or batch file.


In [30]:
import ollama

response = ollama.chat(
    model='mistral',
    messages=[{"role": "user", "content": "Bitte gib einen Titel für das Thema: TikTok und Rezepte"}]
)

print(response['message']['content'])


ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

## 2. Verbesserte Sentiment-Analyse

Die bisher genutzte Methode (`TextBlob`) ist eher für formellere englische Texte geeignet. Hier ergänzen wir zwei modernere Verfahren:

- **VADER**: speziell für Social Media-Texte (auch kurze Aussagen) konzipiert.
- **Transformer-Modell**: z. B. RoBERTa-basiert, feingetunt für Sentiment-Klassifikation auf Twitter-Daten.

In [8]:
from transformers import pipeline

# Sentiment-Pipeline laden (Roberta Twitter-Modell)
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment",
    truncation=True
)

# Einzelne Zeilen durchgehen, um Fehler im Batch zu vermeiden
def safe_sentiment(text):
    try:
        return sentiment_pipeline(text[:512])[0]['label']
    except Exception as e:
        print(f"Fehler bei Text: {text[:100]}... -> {e}")
        return "error"


df_clean['roberta_sentiment'] = df_clean['text_clean'].astype(str).apply(safe_sentiment)

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Fehler bei Text: 𝗙𝗢𝗟𝗘 𝗣𝗨𝗕𝗟𝗜𝗦𝗛𝗜𝗡𝗚 ____________________________ 𝗫𝗵𝗲𝗻𝘀𝗶𝗹𝗮 𝘅 𝗟𝗲𝗱𝗿𝗶 𝗩𝘂𝗹𝗮 𝗠𝗮 𝗞𝘁𝗵𝗲 𝗢𝗳𝗳𝗶𝗰𝗶𝗮𝗹 𝗩𝗶𝗱𝗲𝗼 __________... -> The expanded size of the tensor (582) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 582].  Tensor sizes: [1, 514]
Fehler bei Text: اشترك في قناة حسين الجسمي الرسمية حسين الجسمي فستانك الأبيض استمعوا على جميع المنصات كلمات أمير طعيم... -> The expanded size of the tensor (516) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 516].  Tensor sizes: [1, 514]


## 3. Zeitliche Analyse der Topics

Ein großer Vorteil von BERTopic ist die Möglichkeit, Themen im Zeitverlauf zu analysieren. Dazu müssen wir das Erstellungsdatum korrekt als `datetime` interpretieren und in geeignete Zeitstempel transformieren.

In [9]:
# Sicherstellen, dass timestamp eine gültige datetime-Spalte ist
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'], errors='coerce')
df_clean = df_clean.dropna(subset=['timestamp'])

# Alle Listen synchron extrahieren
texts_synced = df_clean['text_clean'].tolist()
topics_synced = df_clean['topic'].tolist()
timestamps_synced = df_clean['timestamp'].tolist()  # echte datetime-Werte

# Kontrollieren
print(len(texts_synced), len(topics_synced), len(timestamps_synced))

# Topics-over-time berechnen (echte Timestamps → funktioniert!)
topics_over_time = topic_model.topics_over_time(
    docs=texts_synced,
    topics=topics_synced,
    timestamps=timestamps_synced,
    nr_bins=20  # oder eine andere sinnvolle Anzahl Zeit-Buckets
)

# Visualisierung
topic_model.visualize_topics_over_time(topics_over_time)


769 769 769


## 4. Visuelle Exploration mit UMAP

Zur besseren Interpretation der durch BERTopic erzeugten Embeddings und Topics projizieren wir die hochdimensionalen Embeddings in 2D mit **UMAP**. So lassen sich thematische Cluster visuell erkennen.

In [10]:

# Visualisierung nach Topic
topic_model.visualize_documents(texts, topics=topics)

#  Save

In [11]:
df_clean.to_csv(RAW_DIR / "social_media_data_with_topics.csv")


In [12]:
df_clean.head()

source                                                 id  \
1427  youtube         #AborandTynna #OfficialMusicVideo #Baller"   
1428  youtube        #Asche #Kollegah  #BisHierLiefAllesNochGut"   
1429  youtube       #MileyCyrus #MoretoLose #SomethingBeautiful"   
1430  youtube      #Oblivion #ArcRaiders #GameTwo #ZDFNeo #rbtv"   
1431  youtube  #ROSÉ_Messy #F1TheAlbum #F1TheMovie #F1 #ROSÉ ...   

               title                  text username  likes  comments  \
1427    Abor & Tynna  2025-05-08T18:00:06Z   234565    0.0       0.0   
1428           Asche  2025-05-08T22:00:07Z   127947    0.0       0.0   
1429  MileyCyrusVEVO  2025-05-09T04:00:07Z  1029912    0.0       0.0   
1430        Game Two  2025-05-10T10:00:43Z    68615    0.0       0.0   
1431            ROSÉ  2025-05-08T16:00:07Z  7123605    0.0       0.0   

        shares     plays           timestamp         published_at  url  \
1427  0.017715  0.000002 2025-05-12 13:56:29  2025-05-12 13:56:29  NaN   
1428  0.017715  0.000003 2025-05-12 13:56:29  2025-05-12 13:56:29  NaN   
1429  0.017715  0.000015 2025-05-12 13:56:29  2025-05-12 13:56:29  NaN   
1430  0.017715  0.000002 2025-05-12 13:56:29  2025-05-12 13:56:29  NaN   
1431  0.017715  0.000137 2025-05-12 13:56:29  2025-05-12 13:56:29  NaN   

     title_language     title_clean  title_char_count  title_word_count  \
1427             cy      abor tynna              10.0               2.0   
1428             de           asche               5.0               1.0   
1429             tr  mileycyrusvevo              14.0               1.0   
1430             en        game two               8.0               2.0   
1431             de            rosé               4.0               1.0   

      title_uppercase_count  title_exclamation_count  title_question_count  \
1427                    0.0                      0.0                   0.0   
1428                    0.0                      0.0                   0.0   
1429                    0.0                      0.0                   0.0   
1430                    0.0                      0.0                   0.0   
1431                    0.0                      0.0                   0.0   

      title_emoji_count  title_mention_count  title_hashtag_count  \
1427                0.0                  0.0                  0.0   
1428                0.0                  0.0                  0.0   
1429                0.0                  0.0                  0.0   
1430                0.0                  0.0                  0.0   
1431                0.0                  0.0                  0.0   

      title_avg_word_length title_sentiment  title_sentiment_score  \
1427                    4.5         neutral                    0.0   
1428                    5.0         neutral                    0.0   
1429                   14.0         neutral                    0.0   
1430                    3.5        negative                   -0.4   
1431                    4.0         neutral                    0.0   

     text_language text_clean  text_char_count  text_word_count  \
1427            de        t z              3.0              2.0   
1428            de        t z              3.0              2.0   
1429            de        t z              3.0              2.0   
1430            de        t z              3.0              2.0   
1431            de        t z              3.0              2.0   

      text_uppercase_count  text_exclamation_count  text_question_count  \
1427                   0.0                     0.0                  0.0   
1428                   0.0                     0.0                  0.0   
1429                   0.0                     0.0                  0.0   
1430                   0.0                     0.0                  0.0   
1431                   0.0                     0.0                  0.0   

      text_emoji_count  text_mention_count  text_hashtag_count  \
1427               0.0                 0.0  